# 同じ建物 -> 同じ値段のハズ
## なので、testデータのうち、trainデータと同じものがないか調査する

In [128]:
# Import Modules
import os
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
# Enable plotly offline plotting
import plotly
import plotly.graph_objects as go
import seaborn as sns
from IPython.core.display import HTML, display
from ipywidgets import interact


# Ignore sklearn warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

# Set the number of cols/rows max
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

plotly.offline.init_notebook_mode(connected=True)

# Init project path
PROJECT_DIR = os.getcwd() + "/../../"
sys.path.insert(0, PROJECT_DIR)
%load_ext autoreload
%autoreload 2
display(HTML("<style>.container { width:100% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [180]:
from src.exp.exp4_simple.step0_set_global_variables import *
from src.exp.exp4_simple.step1_load_data import load_data

# load data
train_df, test_df = load_data(DATA_DIR=DATA_DIR)

In [181]:
# preprocess
from src.preprocess.address import pp_address
from src.preprocess.age import pp_age
from src.preprocess.floor_stories import pp_floor_stories
train_df = pp_address(train_df)
train_df = pp_age(train_df)
train_df = pp_floor_stories(train_df)
test_df = pp_address(test_df)
test_df = pp_age(test_df)
test_df = pp_floor_stories(test_df)

In [183]:
buildings = train_df[
    ["address_1", "address_2", "age", "stories"]
].drop_duplicates().reset_index().drop("index", axis=1).reset_index().rename(
    columns={"index": "building_id"}
)

In [185]:
train_df = pd.merge(train_df, buildings, on=["address_1", "address_2", "age", "stories"])
test_df = pd.merge(test_df, buildings, on=["address_1", "address_2", "age", "stories"], how="left")

In [198]:
buildings

,building_id,address_1,address_2,age,stories
0,0,北,滝野川,9年9ヶ月,12
1,1,中央,月島,44年10ヶ月,10
2,2,渋谷,笹塚,8年6ヶ月,15
3,3,杉並,高円寺南,29年4ヶ月,4
4,4,葛飾,金町,31年7ヶ月,2
...,...,...,...,...,...
18701,18701,足立,堀之内,16年11ヶ月,3
18702,18702,江東,毛利,13年5ヶ月,3
18703,18703,世田谷,太子堂,39年7ヶ月,4
18704,18704,中野,中野,11年0ヶ月,2


In [209]:
(test_df["building_id"].isnull() == False).sum()

18233